# I. Validating Class

Random verification of n-dimensional Gaussian models in IMQ kernel and KGM kernel.

In [1]:
from stein_q_thinning.target import QTargetIMQ, QTargetAuto, QTargetKGM

import numpy as np
from scipy.stats import wishart
from jax import numpy as jnp

np.random.seed(1234)

In [2]:
dim = 4

x_test = np.random.normal(size=dim)
mean = np.random.normal(size=dim)
cov = wishart.rvs(dim + 10, np.eye(dim), size=1)

print("x_test:", x_test)
print("mean  :", mean)
print("cov   :", cov)

x_test: [ 0.47143516 -1.19097569  1.43270697 -0.3126519 ]
mean  : [-0.72058873  0.88716294  0.85958841 -0.6365235 ]
cov   : [[11.68150068  0.05364739 -7.66509583  3.39029399]
 [ 0.05364739 12.34410368  4.00531349  3.36496236]
 [-7.66509583  4.00531349 25.21091916 -9.90588931]
 [ 3.39029399  3.36496236 -9.90588931 10.75391052]]


In [3]:
# Test manual IMQ class
log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_imq = QTargetIMQ(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv)

print("log-pdf   :", test_manual_imq.log_p(x_test))
print("glog-pdf  :", test_manual_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_imq.hess_log_p(x_test))
print("log-q-pdf :", test_manual_imq.log_q(x_test))
print("glog-q-pdf:", test_manual_imq.grad_log_q(x_test))

log-pdf   : -7.153352187026759
glog-pdf  : [-0.17921809  0.33648779 -0.25339807 -0.31232095]
hlog-pdf  : [[-0.10860438  0.013307   -0.03654333 -0.0035867 ]
 [ 0.013307   -0.12428333  0.05865073  0.08871951]
 [-0.03654333  0.05865073 -0.09838846 -0.09746129]
 [-0.0035867   0.08871951 -0.09746129 -0.20939533]]
log-q-pdf : -7.235925371128592
glog-q-pdf: [-0.13873336  0.23413024 -0.15708047 -0.17007607]


In [4]:
# Test auto IMQ class
log_p = lambda x:-jnp.log(2*jnp.pi) - 0.5*jnp.log(jnp.linalg.det(cov)) - 0.5*(x-mean)@jnp.linalg.inv(cov)@(x-mean)
linv = jnp.linalg.inv(cov)
imq_kernel = lambda x, y: (1 + (x - y)@linv@(x - y))**(-0.5)

test_auto_imq = QTargetAuto(log_p=log_p, reproducing_kernel=imq_kernel)

print("log-pdf   :", test_auto_imq.log_p(x_test))
print("glog-pdf  :", test_auto_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_auto_imq.hess_log_p(x_test))
print("log-q-pdf :", test_auto_imq.log_q(x_test))
print("glog-q-pdf:", test_auto_imq.grad_log_q(x_test))

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


TypeError: QTargetAuto.__init__() got an unexpected keyword argument 'reproducing_kernel'

In [ ]:
# Test manual KGM class
log_p = lambda x:-np.log(2*np.pi) - 0.5*np.log(np.linalg.det(cov)) - 0.5*(x-mean)@np.linalg.inv(cov)@(x-mean)
grad_log_p = lambda x: -np.linalg.inv(cov)@(x-mean)
hess_log_p = lambda x: -np.linalg.inv(cov)
linv = np.linalg.inv(cov)

test_manual_kgm = QTargetKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0)

print("log-pdf   :", test_manual_kgm.log_p(x_test))
print("glog-pdf  :", test_manual_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_kgm.log_q(x_test))
print("glog-q-pdf:", test_manual_kgm.grad_log_q(x_test))

In [ ]:
# Test auto KGM class
s = 3.0
linv = jnp.linalg.inv(cov)
log_p = lambda x:-jnp.log(2*jnp.pi) - 0.5*jnp.log(jnp.linalg.det(cov)) - 0.5*(x-mean)@jnp.linalg.inv(cov)@(x-mean)

kgm_kernel = lambda x, y: (1 + x@linv@x)**((s-1)/2) *\
        (1 + y@linv@y)**((s-1)/2) *\
        (1 + (x-y)@linv@(x-y))**(-0.5) +\
        (1 + x@linv@y )/( jnp.sqrt(1+x@linv@x) * jnp.sqrt(1+y@linv@y) )

test_auto_kgm = QTargetAuto(log_p=log_p, reproducing_kernel=kgm_kernel)

print("log-pdf   :", test_auto_kgm.log_p(x_test))
print("glog-pdf  :", test_auto_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_auto_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_auto_kgm.log_q(x_test))
print("glog-q-pdf:", test_auto_kgm.grad_log_q(x_test))

# II. Verifying Non-Python Objects

Verifying the Availability of Black-Box Log-Posterior Distributions and their Gradients Compiled by Stan

In [ ]:
import os
import json
import pandas as pd
from stein_q_thinning.util import flat
import bridgestan as bs
from posteriordb import PosteriorDatabase

In [ ]:
# Load DataBase Locally
pdb_path = os.path.join(os.getcwd(), "posteriordb/posterior_database")
my_pdb = PosteriorDatabase(pdb_path)

# Load Dataset
posterior = my_pdb.posterior("eight_schools-eight_schools_centered")
stan = posterior.model.stan_code_file_path()
data = json.dumps(posterior.data.values())
model = bs.StanModel.from_stan_file(stan, data)

# Gold Standard
gs = posterior.reference_draws()
df = pd.DataFrame(gs)
gs_chains = np.zeros((sum(flat(posterior.information['dimensions'].values())),\
                       posterior.reference_draws_info()['diagnostics']['ndraws']))
for i in range(len(df.keys())):
    s = []
    for j in range(len(df[df.keys()[i]])):
        s += df[df.keys()[i]][j]
    gs_chains[i, :] = s
linv = np.linalg.inv(np.cov(gs_chains))

# Extract log-P-pdf and its gradient
log_p = model.log_density
grad_log_p = lambda x: model.log_density_gradient(x)[1]
hess_log_p = lambda x: model.log_density_hessian(x)[2]

In [ ]:
# Test manual IMQ class
dim = model.param_num()
x_test = np.random.normal(size=dim)

test_manual_imq = QTargetIMQ(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv)

print("log-pdf   :", test_manual_imq.log_p(x_test))
print("glog-pdf  :", test_manual_imq.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_imq.hess_log_p(x_test))
print("log-q-pdf :", test_manual_imq.log_q(x_test))
print("glog-q-pdf:", test_manual_imq.grad_log_q(x_test))

In [ ]:
# Test manual KGM class
dim = model.param_num()
x_test = np.random.normal(size=dim)

test_manual_kgm = QTargetKGM(log_p=log_p, grad_log_p=grad_log_p, hess_log_p=hess_log_p, linv=linv, s=3.0)

print("log-pdf   :", test_manual_kgm.log_p(x_test))
print("glog-pdf  :", test_manual_kgm.grad_log_p(x_test))
print("hlog-pdf  :", test_manual_kgm.hess_log_p(x_test))
print("log-q-pdf :", test_manual_kgm.log_q(x_test))
print("glog-q-pdf:", test_manual_kgm.grad_log_q(x_test))